<a href="https://colab.research.google.com/github/Sulem1403/ASR/blob/main/Sulem_EulerMotors_ML_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Simulate the dataset
num_samples = 10000
ambient_temp = np.random.uniform(20, 35, num_samples)

data = {
    'RPM': np.random.randint(0, 5001, num_samples),
    'AC_Motor_Current': np.random.uniform(0, 100, num_samples),
    'Throttle_Command': np.random.randint(0, 10, num_samples),
    'Motor_Temperature': ambient_temp + np.random.uniform(0, 40, num_samples),
    'Battery_Temperature': ambient_temp + np.random.uniform(0, 20, num_samples),
}

In [ ]:
df = pd.DataFrame(data)


In [ ]:
df['AC_Motor_Current'] = df['RPM'] * 0.02 + np.random.normal(0, 5, num_samples)
df['Motor_Temperature'] = df['AC_Motor_Current'] * 0.5 + ambient_temp + np.random.normal(0, 5, num_samples)
df['Battery_Temperature'] = df['AC_Motor_Current'] * 0.3 + ambient_temp + np.random.normal(0, 3, num_samples)


In [ ]:
# Add ambient temperature to the DataFrame
df['Ambient_Temperature'] = ambient_temp

In [ ]:
# Define labels for overload, normal load, underload
df['Load_Condition'] = pd.cut(df['RPM'], bins=[-np.inf, 1000, 4000, np.inf], labels=['underload', 'normal', 'overload'])


In [ ]:
# Define motor and battery degradation
motor_temp_threshold = 80  # Example threshold
battery_temp_threshold = 60  # Example threshold

df['Motor_Degradation'] = df['Motor_Temperature'] > motor_temp_threshold
df['Battery_Degradation'] = df['Battery_Temperature'] > battery_temp_threshold


In [ ]:
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Battery_Temperature', 'Ambient_Temperature']])


In [ ]:
# Dimensionality reduction with PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(scaled_data)

In [ ]:
# Clustering with K-means
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(pca_data)
df['Cluster'] = clusters

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Rank clusters based on fleet average performance (excluding categorical columns)
cluster_means = df.groupby('Cluster')[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Battery_Temperature']].mean()
performance_metric = cluster_means.mean(axis=1)
cluster_ranks = performance_metric.sort_values(ascending=False).reset_index()
cluster_ranks['Rank'] = range(1, len(cluster_ranks) + 1)
cluster_rank_dict = cluster_ranks.set_index('Cluster')['Rank'].to_dict()
df['Cluster_Rank'] = df['Cluster'].map(cluster_rank_dict)

In [ ]:
# Prepare data for load condition classification
X_load = df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Battery_Temperature', 'Ambient_Temperature']]
y_load = df['Load_Condition']

In [ ]:
X_train_load, X_test_load, y_train_load, y_test_load = train_test_split(X_load, y_load, test_size=0.2, random_state=42)



In [ ]:
# Train classifier for load condition
load_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
load_classifier.fit(X_train_load, y_train_load)


RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate classifier for load condition
y_pred_load = load_classifier.predict(X_test_load)
print("Load Condition Classification Report:")
print(classification_report(y_test_load, y_pred_load))

Load Condition Classification Report:
              precision    recall  f1-score   support

      normal       1.00      1.00      1.00      1220
    overload       1.00      1.00      1.00       380
   underload       1.00      1.00      1.00       400

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
# Prepare data for motor degradation classification
X_motor = df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Ambient_Temperature']]
y_motor = df['Motor_Degradation']

X_train_motor, X_test_motor, y_train_motor, y_test_motor = train_test_split(X_motor, y_motor, test_size=0.2, random_state=42)


In [ ]:
# Train classifier for motor degradation
motor_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
motor_classifier.fit(X_train_motor, y_train_motor)

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate classifier for motor degradation
y_pred_motor = motor_classifier.predict(X_test_motor)
print("Motor Degradation Classification Report:")
print(classification_report(y_test_motor, y_pred_motor))

Motor Degradation Classification Report:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1940
        True       1.00      1.00      1.00        60

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
# Prepare data for battery degradation classification
X_battery = df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Battery_Temperature', 'Ambient_Temperature']]
y_battery = df['Battery_Degradation']

X_train_battery, X_test_battery, y_train_battery, y_test_battery = train_test_split(X_battery, y_battery, test_size=0.2, random_state=42)


In [ ]:
# Train classifier for battery degradation
battery_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
battery_classifier.fit(X_train_battery, y_train_battery)


RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate classifier for battery degradation
y_pred_battery = battery_classifier.predict(X_test_battery)
print("Battery Degradation Classification Report:")
print(classification_report(y_test_battery, y_pred_battery))

Battery Degradation Classification Report:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1937
        True       1.00      1.00      1.00        63

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [ ]:
# New vehicle data
new_vehicle = np.array([[3000, 50, 7, 75, 55, 30]])  # Including ambient temperature
new_vehicle_df = pd.DataFrame(new_vehicle, columns=['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Battery_Temperature', 'Ambient_Temperature'])


In [ ]:
# Standardize the new vehicle data
new_vehicle_scaled = scaler.transform(new_vehicle_df)


In [ ]:
# Apply PCA transformation
new_vehicle_pca = pca.transform(new_vehicle_scaled)

In [ ]:
# Predict the load condition for the new vehicle
new_vehicle_load_condition = load_classifier.predict(new_vehicle_df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Battery_Temperature', 'Ambient_Temperature']])
print(f'The new vehicle is predicted to be in load condition: {new_vehicle_load_condition[0]}')


The new vehicle is predicted to be in load condition: normal


In [ ]:
# Predict the motor degradation for the new vehicle
new_vehicle_motor_degradation = motor_classifier.predict(new_vehicle_df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Motor_Temperature', 'Ambient_Temperature']])
print(f'The new vehicle motor degradation is: {new_vehicle_motor_degradation[0]}')


The new vehicle motor degradation is: False


In [ ]:
# Predict the battery degradation for the new vehicle
new_vehicle_battery_degradation = battery_classifier.predict(new_vehicle_df[['RPM', 'AC_Motor_Current', 'Throttle_Command', 'Battery_Temperature', 'Ambient_Temperature']])
print(f'The new vehicle battery degradation is: {new_vehicle_battery_degradation[0]}')

The new vehicle battery degradation is: False


In [ ]:
# Predict the cluster for the new vehicle
new_vehicle_cluster = kmeans.predict(new_vehicle_pca)
new_vehicle_cluster_rank = cluster_rank_dict[new_vehicle_cluster[0]]
print(f'The new vehicle is predicted to be in cluster: {new_vehicle_cluster[0]}, which has rank: {new_vehicle_cluster_rank}')

The new vehicle is predicted to be in cluster: 0, which has rank: 2
